# Model Accuracy

Up until now, I've been using mostly qualitative, visual assessment to determine how well a model performed. I want to create a better metric. I'm thinking that a good metric will be the percentage of pixels correctly labelled; let's see how well it works.

In [1]:
import torch
import pandas as pd

import monai
from monai.transforms import (
    Compose,
    LoadNiftid,
    NormalizeIntensityd,
    AddChanneld,
    ToTensord,
    CenterSpatialCropd,
    ScaleIntensityRangePercentilesd,
)
from monai.metrics import DiceMetric
from monai.inferers import sliding_window_inference

import numpy as np

In [2]:
def get_model_metrics(model):
    data_dir = 'data/new/'
    test_imgs = []
    with open(data_dir + 'test_imgs.txt', 'r') as f:
        test_imgs = [data_dir + image.rstrip() for image in f.readlines()]

    test_masks = []
    with open(data_dir + 'test_masks.txt', 'r') as f:
        test_masks = [data_dir + mask.rstrip() for mask in f.readlines()]

    test_dicts = [{'image': image, 'mask': mask} for (image, mask) in zip(test_imgs, test_masks)]
    

    # Basic transforms
    data_keys = ["image", "mask"]
    data_transforms = Compose(
        [
            LoadNiftid(keys=data_keys),
            AddChanneld(keys=data_keys),
#             NormalizeIntensityd(keys="image"),
            ScaleIntensityRangePercentilesd(
                keys='image',
                lower=25,
                upper=75,
                b_min=-0.5,
                b_max=0.5
            )
        ]
    )

    test_dataset = monai.data.CacheDataset(
        data=test_dicts,
        transform=Compose(
            [
                data_transforms,
                ToTensord(keys=data_keys)
            ]
        ),
        cache_rate=1.0
    )
    
    accuracies = np.array([])
    
    metrics = np.array([])
    dice_metric = DiceMetric(
        include_background=True,
    )
    
    for sample in test_dataset:
        test_image = sample['image'].unsqueeze(0).cuda(1)
        
        test_mask = sliding_window_inference(
            test_image,
            roi_size=[256, 256, 16],
            sw_batch_size=1,
            predictor=model,
            device=torch.device('cuda:1')
        )
        
        test_mask = test_mask.argmax(1).cpu()
        
        metric = dice_metric(
            y_pred=test_mask.unsqueeze(0),
            y=sample['mask'].unsqueeze(0)
        )
        
        metrics = np.append(metrics, metric)
        accuracy = 1 - torch.sum(torch.abs(sample['mask'] - test_mask)) / sample['mask'].numel()
        accuracies = np.append(accuracies, accuracy)
        
    avg_accuracy = np.average(accuracies)
    median_accuracy = np.median(accuracies)
    avg_metric = np.average(metrics)
    
    return {
        'average': avg_accuracy,
        'median': median_accuracy,
        'dice_metric': avg_metric
    }

In [3]:
import glob
import CustomModels
from lightning_modules import UNet3D, UNet3D_Large

In [4]:
def print_acc_metrics(model_class, checkpoint_path):
    model = model_class.load_from_checkpoint(checkpoint_path).cuda(1)
    model.eval()
    print(model.hparams.name)
    metrics = get_model_metrics(model)
    print('Average Accuracy: ', metrics['average'])
    print('Median Accuracy: ', metrics['median'])
    print('Average Dice: ', metrics['dice_metric'])

## 3D UNet

In [ ]:
print_acc_metrics(CustomModels.UNet_3D, 'models/7-15-2020_3dremastered/_ckpt_epoch_99.ckpt')

## Dice + Focal Loss

In [ ]:
print_acc_metrics(CustomModels.UNet_DF, 'models/7-8-2020_dicefocal/_ckpt_epoch_119.ckpt')

## MaskGAN

In [ ]:
print_acc_metrics(CustomModels.MaskGAN, 'models/7-16-2020_MaskGAN2/_ckpt_epoch_234.ckpt')

## With Residuals

In [6]:
print_acc_metrics(UNet3D.UNet3D, 'models/2020-10-14_UNet_NewData/epoch=89-val_loss=0.10.ckpt')

Load and cache transformed data: 100%|██████████| 80/80 [00:00<00:00, 140160.53it/s]

2020-10-14_UNet_NewData


Average Accuracy:  0.9921809174120426
Median Accuracy:  0.9926690459251404
Average Dice:  0.8770074814558029


In [5]:
print_acc_metrics(UNet3D_Large.UNet3D, 'models/2020-10-15_UNet_MoreParams/epoch=74-val_loss=0.10.ckpt')

Load and cache transformed data: 100%|██████████| 80/80 [00:00<00:00, 118692.72it/s]

2020-10-14_UNet_NewData


Average Accuracy:  0.9924753688275814
Median Accuracy:  0.9928219616413116
Average Dice:  0.8834207907319069
